<a href="https://colab.research.google.com/github/SupermarketAutomationAI/data_processing/blob/main/V1_seperatedDS_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading, Splitting, and Processing

This Jupyter notebook contains the data processing for the first version of data processing. The format of this version is as follows:

/SupermarketAutomationAI_Dataset_V1
```
/train
  Multiple Bananas
  One Banana
  .
  .
  .
/val
  Multiple Bananas
  One Banana
  .
  .
  .
/test
  Multiple Bananas
  One Banana
  .
  .
  .
```
Where each of the above subdirectories in train, val, and test are their own class. In this version, there are 14 classes to classify.

Classes: 

['Multiple Bananas', 'Multiple Fuji Apples', 'Multiple Gala Apples', 'Multiple Golden Delicious Apples', 'Multiple Granny Smith Apples', 'Multiple Oranges', 'Multiple Red Delicious Apples', 'One Banana', 'One Fuji Apple', 'One Gala Apple', 'One Golden Delicious Apple', 'One Granny Smith Apple', 'One Orange', 'One Red Delicious Apple']

## Load the Data

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip '/content/drive/My Drive/APS360 Project/Dataset Zip Files/SupermarketAutomationAI_Dataset_V1' -d '/root/datasets'

In [ ]:
# Clean the data, remove "._" from file names
import os
from glob import glob
parent_dir = "/root/datasets/SupermarketAutomationAI_Dataset_V1"
classes = ['Multiple Bananas', 'Multiple Fuji Apples', 'Multiple Gala Apples', 'Multiple Golden Delicious Apples', 
           'Multiple Granny Smith Apples', 'Multiple Oranges', 'Multiple Red Delicious Apples', 'One Banana', 
           'One Fuji Apple', 'One Gala Apple', 'One Golden Delicious Apple', 'One Granny Smith Apple', 'One Orange', 
           'One Red Delicious Apple']

for cat in classes:
    path = parent_dir + '/' + cat
    invalid_files = glob(os.path.join(path,"._*"))
    for file_ in invalid_files:
        os.rename(file_, os.path.join(path,file_.split('/')[-1].replace("._","")))

## Split the data

In [ ]:
# Import the necessary libraries
import json
import os
import math
import shutil

In [ ]:
# create data directory and move all images into it
parent_dir = "/root/datasets/SupermarketAutomationAI_Dataset_V1"
os.chdir(parent_dir)
category_list = list(filter(lambda x: os.path.isdir(x), os.listdir()))
data_dir = parent_dir + '/' + "data"
os.mkdir(data_dir, 755)
for category in category_list:
    cat_dir = parent_dir + '/' + category
    shutil.move(cat_dir, data_dir)

In [ ]:
train_split = 0.6

dataset_dirs= ['train','val','test']
for dsdirs in dataset_dirs:
 path = parent_dir + '/'+ dsdirs
 os.mkdir( path,755 )

for category in category_list: 
    src_path = parent_dir + '/data/' + category
    train_dir = parent_dir + '/train/' + category + '/'
    val_dir = parent_dir + '/val/' + category + '/'
    test_dir = parent_dir + '/test/' + category + '/'
    
    os.mkdir(train_dir, 755 )
    os.mkdir(val_dir, 755)
    os.mkdir(test_dir, 755)

    #get files' names list from respective directories
    os.chdir(src_path)
    files = [f for f in os.listdir() if os.path.isfile(f)]

    #get training, testing and validation files count
    train_count = math.ceil(train_split*len(files))
    valid_count = int((len(files)-train_count)/2)
    test_count = valid_count

    #get files to segragate for train,test and validation data set
    train_data_list = files[0: train_count-1]
    valid_data_list = files[train_count:train_count+valid_count-1] 
    test_data_list = files[train_count+valid_count:]


    for train_data in train_data_list:
        train_path = src_path + '/' + train_data
        shutil.move(train_path,train_dir)

    for valid_data in valid_data_list:
        valid_path = src_path + '/' + valid_data
        shutil.move(valid_path,val_dir)

    for test_data in test_data_list:
        test_path = src_path + '/' + test_data
        shutil.move(test_path,test_dir)

    # Move any files that are left behind into the training directory
    os.chdir(src_path)
    files = [f for f in os.listdir() if os.path.isfile(f)]
    for img_left_behind in files:
        img_path = src_path + '/' + img_left_behind
        shutil.move(img_path, train_dir)

## Preprocess data into Dataloaders

In [ ]:
# import needed libraries for this section
import torch
from torchvision import datasets, transforms

In [ ]:
# define the locations of the training and validation data
train_dir = os.path.join(parent_dir, 'train/')
val_dir = os.path.join(parent_dir, 'val/')
test_dir = os.path.join(parent_dir, 'test/')

# define a list of all classes that the model will be trained with

# List different classes: 14 currently
classes = ['Multiple Bananas', 'Multiple Fuji Apples', 'Multiple Gala Apples', 'Multiple Golden Delicious Apples', 
           'Multiple Granny Smith Apples', 'Multiple Oranges', 'Multiple Red Delicious Apples',
           'One Banana', 'One Fuji Apple', 'One Gala Apple', 'One Golden Delicious Apple',
           'One Granny Smith Apple', 'One Orange', 'One Red Delicious Apple']

# three possible transforms, will test each to see which gives better results
data_CC_transform = transforms.Compose([transforms.CenterCrop([224,224]), transforms.ToTensor()])

# apply the transforms to the data
train_data = datasets.ImageFolder(train_dir, transform=data_CC_transform)
val_data = datasets.ImageFolder(val_dir, transform=data_CC_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_CC_transform)

# print the amount of data in both training and validation sets
print("Amount of training data: ", len(train_data))
print("Amount of validation data: ", len(val_data))
print("Amount of test data: ", len(test_data))

Amount of training data:  6936
Amount of validation data:  2287
Amount of test data:  2305


# Model Building, Training, and Testing